In [34]:
import pandas as pd
import statsmodels.api as sm
# import numpy as np

In [35]:
# Load the data
df = pd.read_csv('./walmart_data.csv')
df.columns = [c.lower() for c in df.columns]
df.head()

def get_agg_df():
    user_agg_df = df.groupby(['user_id']).agg({
        'product_id': 'count',
        'purchase': 'sum',
        'occupation': 'min',
        'gender': 'min',
        'age': 'min',
        'city_category': 'min',
        'stay_in_current_city_years': 'min',
        'marital_status': 'min',
    }).reset_index()
    columns = user_agg_df.columns
    return user_agg_df, columns

user_agg_df, columns = get_agg_df()

user_agg_df.head()

,user_id,product_id,purchase,occupation,gender,age,city_category,stay_in_current_city_years,marital_status
0,1000001,35,334093,10,F,0-17,A,2,0
1,1000002,77,810472,16,M,55+,C,4+,0
2,1000003,29,341635,15,M,26-35,A,3,0
3,1000004,14,206468,7,M,46-50,B,2,1
4,1000005,106,821001,20,M,26-35,A,1,1


In [36]:
#first comparing gender and age to predict expenditure per individual. both gender and age use one hot encoding
user_agg_df, columns = get_agg_df()
features = ['gender', 'age']
df_encoded = pd.get_dummies(user_agg_df, columns=features, drop_first=True)

Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in features + ['product_id']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   product_id  gender_M  age_18-25  age_26-35  age_36-45  age_46-50  \
0        35.0       0.0        0.0        0.0        0.0        0.0   
1        77.0       1.0        0.0        0.0        0.0        0.0   
2        29.0       1.0        0.0        1.0        0.0        0.0   
3        14.0       1.0        0.0        0.0        0.0        1.0   
4       106.0       1.0        0.0        1.0        0.0        0.0   

   age_51-55  age_55+  
0        0.0      0.0  
1        0.0      1.0  
2        0.0      0.0  
3        0.0      0.0  
4        0.0      0.0  
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.960
Method:                 Least Squares   F-statistic:                 1.749e+04
Date:                Thu, 20 Feb 2025   Prob (F-statistic):               0.00
Time:                        10:24

In [37]:
#second, due to warning in strong multicollinearity, attempting to use label encoding for age

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender']
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['product_id']+ ['encoded_age']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   product_id  gender_M  encoded_age
0        35.0       0.0          0.0
1        77.0       1.0          6.0
2        29.0       1.0          2.0
3        14.0       1.0          4.0
4       106.0       1.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                 4.651e+04
Date:                Thu, 20 Feb 2025   Prob (F-statistic):               0.00
Time:                        10:24:07   Log-Likelihood:                -79958.
No. Observations:                5891   AIC:                         1.599e+05
Df Residuals:                    5887   BIC:                         1.600e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                              

In [38]:
#third, exclude product id

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender']
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   gender_M  encoded_age
0       0.0          0.0
1       1.0          6.0
2       1.0          2.0
3       1.0          4.0
4       1.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     44.65
Date:                Thu, 20 Feb 2025   Prob (F-statistic):           5.67e-20
Time:                        10:24:09   Log-Likelihood:                -89360.
No. Observations:                5891   AIC:                         1.787e+05
Df Residuals:                    5888   BIC:                         1.787e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|     

In [39]:
#fourth, include occupation

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender']
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age', 'occupation']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   occupation  gender_M  encoded_age
0        10.0       0.0          0.0
1        16.0       1.0          6.0
2        15.0       1.0          2.0
3         7.0       1.0          4.0
4        20.0       1.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     30.01
Date:                Thu, 20 Feb 2025   Prob (F-statistic):           3.03e-19
Time:                        10:24:27   Log-Likelihood:                -89359.
No. Observations:                5891   AIC:                         1.787e+05
Df Residuals:                    5887   BIC:                         1.788e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                              

In [40]:
#5, age, gender, city

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender', "city_category"]
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   gender_M  city_category_B  city_category_C  encoded_age
0       0.0              0.0              0.0          0.0
1       1.0              0.0              1.0          6.0
2       1.0              0.0              0.0          2.0
3       1.0              1.0              0.0          4.0
4       1.0              0.0              0.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 20 Feb 2025   Prob (F-statistic):          5.05e-215
Time:                        10:27:05   Log-Likelihood:                -88904.
No. Observations:                5891   AIC:                         1.778e+05
Df Residuals:                    5886   BIC:                         1.779e+05
Df Model:     

In [ ]:
#6, age, gender, marital status

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender', "marital_status"]
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   gender_M  marital_status_1  encoded_age
0       0.0               0.0          0.0
1       1.0               0.0          6.0
2       1.0               0.0          2.0
3       1.0               1.0          4.0
4       1.0               1.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     29.80
Date:                Thu, 20 Feb 2025   Prob (F-statistic):           4.08e-19
Time:                        10:29:18   Log-Likelihood:                -89360.
No. Observations:                5891   AIC:                         1.787e+05
Df Residuals:                    5887   BIC:                         1.788e+05
Df Model:                           3                                         
Covariance Type:            non

In [45]:
#7, age, gender, stay in current city years

user_agg_df, columns = get_agg_df()

one_hot_features = ['gender']
label_encoding_features = ['age', "stay_in_current_city_years"]
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)
# print(df_encoded["stay_in_current_city_years"].unique())

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)
stay_years_map = {'2': 2, '0': 0, '1': 1, '3': 3, '4+': 4}
df_encoded['stay_encoded'] = df_encoded['stay_in_current_city_years'].map(stay_years_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age', 'stay_encoded']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())


   gender_M  encoded_age  stay_encoded
0       0.0          0.0           2.0
1       1.0          6.0           4.0
2       1.0          2.0           3.0
3       1.0          4.0           2.0
4       1.0          2.0           1.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     29.76
Date:                Thu, 20 Feb 2025   Prob (F-statistic):           4.31e-19
Time:                        10:35:29   Log-Likelihood:                -89360.
No. Observations:                5891   AIC:                         1.787e+05
Df Residuals:                    5887   BIC:                         1.788e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                  

In [ ]:
#8, age, city

user_agg_df, columns = get_agg_df()

one_hot_features = ["city_category"]
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features + ['encoded_age']]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   city_category_B  city_category_C  encoded_age
0              0.0              0.0          0.0
1              0.0              1.0          6.0
2              0.0              0.0          2.0
3              1.0              0.0          4.0
4              0.0              0.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     331.9
Date:                Thu, 20 Feb 2025   Prob (F-statistic):          4.18e-199
Time:                        10:42:22   Log-Likelihood:                -88944.
No. Observations:                5891   AIC:                         1.779e+05
Df Residuals:                    5887   BIC:                         1.779e+05
Df Model:                           3                                     

In [47]:
#9, city

user_agg_df, columns = get_agg_df()

one_hot_features = ["city_category"]
label_encoding_features = ['age']
df_encoded = pd.get_dummies(user_agg_df, columns=one_hot_features, drop_first=True)

age_map = {'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6}
df_encoded['encoded_age'] = df_encoded['age'].map(age_map)


Y = df_encoded['purchase']
X = df_encoded.drop(columns=[c for c in columns if c not in one_hot_features]).astype(float) # product_id is th enumber of proudcts
print(X.head())
X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

   city_category_B  city_category_C  encoded_age
0              0.0              0.0          0.0
1              0.0              1.0          6.0
2              0.0              0.0          2.0
3              1.0              0.0          4.0
4              0.0              0.0          2.0
                            OLS Regression Results                            
Dep. Variable:               purchase   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     331.9
Date:                Thu, 20 Feb 2025   Prob (F-statistic):          4.18e-199
Time:                        10:45:48   Log-Likelihood:                -88944.
No. Observations:                5891   AIC:                         1.779e+05
Df Residuals:                    5887   BIC:                         1.779e+05
Df Model:                           3                                     